# TlF ground state spectrum

For the X state, we use the Hamiltonian from Table 1 of [D.A. Wilkening, N.F. Ramsey, and D.J. Larson, Phys Rev A **29**, 425 (1984)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.29.425).

For the B state, the Hamiltonian was derived by Oskari Timgren.

In [1]:
import pickle
import numpy as np
from tqdm.notebook import tqdm
from sympy.physics.wigner import wigner_3j

from utils.states import CoupledBasisState, UncoupledBasisState, State

### Units and constants

For the X state:

In [ ]:
Jmax = 6      # max J value in Hamiltonian
I_Tl = 1/2    # I1 in Ramsey's notation
I_F = 1/2     # I2 in Ramsey's notation

# TlF constants. Data from D.A. Wilkening, N.F. Ramsey,
# and D.J. Larson, Phys Rev A 29, 425 (1984). Everything in Hz.

B_rot_X = 6689920000 # superseeded below
c1 = 126030.0
c2 = 17890.0
c3 = 700.0
c4 = -13300.0

D_TlF = 4.2282 * 0.393430307 *5.291772e-9/4.135667e-15 # [Hz/(V/cm)]

# Constants from Wilkening et al, in Hz/Gauss, for 205Tl

mu_J = 35
mu_Tl = 1240.5
mu_F = 2003.63

# Values for rotational constant are from "Microwave Spectral tables: Diatomic molecules" by Lovas & Tiemann (1974). 
# Note that Brot differs from the one given by Ramsey by about 30 MHz.

B_ϵ = 6.689873e9
α = 45.0843e6
B_rot_X = B_ϵ - α/2

For the B state:

In [ ]:
Jmin = 1
Omegas = [1, -1]
B_rot_B = 6687.879e6
D_rot_B = 0.010869e6
H_const_B = -8.1e-2
h1_Tl = 28789e6
h1_F = 861e6
q = 2.423e6
c_Tl = -7.83e6
c1p_Tl = 11.17e6
μ_B = 100
gL = 1
gS = 2

### Representing the states

A state, in general, can be written as a weighted superposition of the basis states. Some of the operations we can define on the basis states are:

- construction
- equality testing;
- inner product, returning either 0 or 1;
- superposition and scalar multiplication, returning a `State` object
- a convenience function to print out all quantum numbers

In particular, two bases are useful when dealing with TlF molecules. The so-called coupled basis consists of kets of form $|F, m_F, F_1, J, I_1, I_2, \Omega\rangle$, whereas the uncoupled basis consists of kets of form $|J, m_J, I_1, m_1, I_2, m_2\rangle$.

A general state `State` can have any number of components, so let's represent it as an list of pairs `(amp, psi)`, where `amp` is the relative amplitude of a component, and `psi` is a basis state. The same component must not appear twice on the list.

There are three operations we can define on the states:

- construction
- superposition: concatenate component arrays and return a `State`
- scalar multiplication `a * psi` and `psi * a`, division, negation
- component-wise inner product `psi1 @ psi2`, where `psi1` is a bra, and `psi2` a ket, returning a complex number

In addition, I define an iterator method to loop through the components, and the `__getitem__()` method to access the components (which are not necessarily in any particular order!). See [Classes/Iterators](https://docs.python.org/3/tutorial/classes.html#iterators) for details.

The state classes are defined in `utils/states.py` (already imported above).

### Operators in Python

Define QM operators as Python functions that take `BasisState` objects, and return `State` objects. Since we are interested in finding matrix elements, we only need the action of operators on the basis states (but it'd be easy to generalize using a `for` loop).

The easiest operators to define are the diagonal ones $J^2, J^4, J^6, J_z, I_{1z}, I_{2z}$, which just multiply the state by their eigenvalue:

In [4]:
def J2(psi):
    return State([(psi.J*(psi.J+1),psi)])

def J4(psi):
    return State([( (psi.J*(psi.J+1))**2, psi)])

def J6(psi):
    return State([( (psi.J*(psi.J+1))**3, psi)])

def Jz(psi):
    return State([(psi.mJ,psi)])

def I1z(psi):
    return State([(psi.m1,psi)])

def I2z(psi):
    return State([(psi.m2,psi)])

The other angular momentum operators we can obtain through the ladder operators

$$ J_\pm=J_x\pm iJ_y. $$

These are defined through their action on the basis states as (Sakurai eqns 3.5.39-40)

$$ J_\pm|J,m\rangle=\sqrt{(j\mp m)(j\pm m+1)}|jm\pm1\rangle. $$

Similarly, $I_{1\pm},I_{2\pm}$ act on the $|I_1,m_1\rangle$ and $|I_2,m_2\rangle$ subspaces in the same way.

In [5]:
def Jp(psi):
    amp = np.sqrt((psi.J-psi.mJ)*(psi.J+psi.mJ+1))
    ket = UncoupledBasisState(psi.J, psi.mJ+1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp,ket)])

def Jm(psi):
    amp = np.sqrt((psi.J+psi.mJ)*(psi.J-psi.mJ+1))
    ket = UncoupledBasisState(psi.J, psi.mJ-1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I1p(psi):
    amp = np.sqrt((psi.I1-psi.m1)*(psi.I1+psi.m1+1))
    ket = UncoupledBasisState(psi.J, psi.mJ, psi.I1, psi.m1+1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I1m(psi):
    amp = np.sqrt((psi.I1+psi.m1)*(psi.I1-psi.m1+1))
    ket = UncoupledBasisState(psi.J, psi.mJ, psi.I1, psi.m1-1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I2p(psi):
    amp = np.sqrt((psi.I2-psi.m2)*(psi.I2+psi.m2+1))
    ket = UncoupledBasisState(psi.J, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2+1)
    return State([(amp,ket)])

def I2m(psi):
    amp = np.sqrt((psi.I2+psi.m2)*(psi.I2-psi.m2+1))
    ket = UncoupledBasisState(psi.J, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2-1)
    return State([(amp,ket)])

In terms of the above-defined ladder operators, we can write

$$J_x=\frac{1}{2}(J_++J_-);\quad
J_y=\frac{1}{2i}(J_+-J_-),$$

and similarly for $I_{1x}, I_{1y}$ and $I_{2x}, I_{2y}$.

In [6]:
def Jx(psi):
    return (1/2)*( Jp(psi) + Jm(psi) )

def Jy(psi):
    return -(1/2)*1j*( Jp(psi) - Jm(psi) )

def I1x(psi):
    return (1/2)*( I1p(psi) + I1m(psi) )

def I1y(psi):
    return -(1/2)*1j*( I1p(psi) - I1m(psi) )

def I2x(psi):
    return (1/2)*( I2p(psi) + I2m(psi) )

def I2y(psi):
    return -(1/2)*1j*( I2p(psi) - I2m(psi) )

### Composition of operators

All operators defined above can only accept `UncoupledBasisStates` as their inputs, and they all return `States` as output. To allow composition of operators,

$$\hat A\hat B|\psi\rangle=\hat A(\hat B(|\psi\rangle)),$$

define the following function.

In [7]:
def com(A, B, psi):
    ABpsi = State()
    # operate with A on all components in B|psi>
    for amp,cpt in B(psi):
        ABpsi += amp * A(cpt)
    return ABpsi

### Rotational term

The simplest term in the Hamiltonian simply gives the rotational levels:

$$H_\text{rot}=B_\text{rot}\vec J^2.$$

In [8]:
def Hrot_X(psi):
    return B_rot_X * J2(psi)

For the B state, the expression is

In [9]:
def Hrot_B(psi):        
    return B_rot_B * J2(psi) - D_rot_B * J4(psi) + H_const_B * J6(psi)

### Terms with ang. momentum dot products

Note that the dot product of two angular momentum operators can be written in terms of the ladder operators as

$$\vec A\cdot\vec B=A_zB_z+\frac{1}{2}(A_+B_-+A_-B_+).$$

We have the following terms (from Table 1 of Ramsey's paper):

$$
H_\text{c1}=c_1\vec I_1\cdot\vec J;\quad
H_\text{c2}=c_2\vec I_2\cdot\vec J;\quad
H_\text{c4}=c_4\vec I_1\cdot\vec I_2\\
H_\text{c3a}=15c_3\frac{(\vec I_1\cdot\vec J)(\vec I_2\cdot\vec J)}{(2J+3)(2J-1)}
=\frac{15c_3}{c_1c_2}\frac{H_\text{c1}H_\text{c2}}{(2J+3)(2J-1)}\\
H_\text{c3b}=15c_3\frac{(\vec I_2\cdot\vec J)(\vec I_1\cdot\vec J)}{(2J+3)(2J-1)}
=\frac{15c_3}{c_1c_2}\frac{H_\text{c2}H_\text{c1}}{(2J+3)(2J-1)}\\
H_\text{c3c}=-10c_3\frac{(\vec I_1\cdot\vec I_2)\vec J^2}{(2J+3)(2J-1)}
=\frac{-10c_3}{c_4 B_\text{rot}}\frac{H_\text{c4}H_\text{rot}}{(2J+3)(2J-1)}
$$

In [10]:
def Hc1(psi):
    return c1 * ( com(I1z,Jz,psi) + (1/2)*(com(I1p,Jm,psi)+com(I1m,Jp,psi)) )

def Hc2(psi):
    return c2 * ( com(I2z,Jz,psi) + (1/2)*(com(I2p,Jm,psi)+com(I2m,Jp,psi)) )

def Hc4(psi):
    return c4 * ( com(I1z,I2z,psi) + (1/2)*(com(I1p,I2m,psi)+com(I1m,I2p,psi)) )

def Hc3a(psi):
    return 15*c3/c1/c2 * com(Hc1,Hc2,psi) / ((2*psi.J+3)*(2*psi.J-1))

def Hc3b(psi):
    return 15*c3/c1/c2 * com(Hc2,Hc1,psi) / ((2*psi.J+3)*(2*psi.J-1))

def Hc3c(psi):
    return -10*c3/c4/B_rot_X * com(Hc4,Hrot_X,psi) / ((2*psi.J+3)*(2*psi.J-1))

The overall field-free Hamiltonian for the X state is

In [11]:
def Hff_X(psi):
    return Hrot_X(psi) + Hc1(psi) + Hc2(psi) + Hc3a(psi) + Hc3b(psi) \
            + Hc3c(psi) + Hc4(psi)

### $\Lambda$-doubling term
Couples terms with opposite values of $\Omega$ and shifts e-parity terms up in energy and f-parity down in energy

In [12]:
def H_LD(psi):
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    Omega = psi.Omega
    Omegaprime = -Omega
    
    amp = (q*(-1)**(J-Omegaprime)/(2*np.sqrt(6)) * wigner_3j(J,2,J,-Omegaprime,Omegaprime-Omega, Omega)
           *np.sqrt((2*J-1)*2*J*(2*J+1)*(2*J+2)*(2*J+3)) )
    ket = UncoupledBasisState(J, mJ, I1, m1, I2, m2, Omegaprime)
    
    return State([(amp,ket)])

### C'(Tl) - term
Coding up the C'(Tl) -term from Brown 1978 "A determination of fundamental Zeeman parameters for the OH radical".

First as given by Brown in eqn A12:

In [13]:
def H_c1p(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    Omega = psi.Omega
    
    #I1, I2 and m2 must be the same for non-zero matrix element
    I1prime = I1
    m2prime = m2
    I2prime = I2
    
    #To have non-zero matrix element need OmegaPrime = -Omega
    Omegaprime = -Omega
    
    #q is chosen such that q == Omegaprime
    q = Omega
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over the possible values of quantum numbers for which the matrix element can be non-zero
    #Need Jprime = J+1 ... |J-1|
    for Jprime in range(np.abs(J-1), J+2):    
        #Loop over possible values of mJprime and m1prime
        for mJprime in np.arange(-Jprime,Jprime+1):
            #Must have mJ+m1 = mJprime + m1prime
            m1prime = mJ+m1-mJprime
            if np.abs(m1prime <= I1):
                #Evaluate the matrix element

                #Matrix element for T(J)T(I)
                term1 = ((-1)**(Jprime-Omegaprime+I1-m1-q+mJprime)*np.sqrt(Jprime*(Jprime+1)*(2*Jprime+1)**2*(2*J+1)*I1*(I1+1)
                                                                           *(2*I1+1))
                         * complex(wigner_3j(Jprime,1,J,-mJprime,mJprime-mJ, mJ)) * complex(wigner_3j(I1,1,I1,-m1prime, m1prime-m1, m1))
                         * complex(wigner_3j(Jprime, 1, J, 0, -q, Omega)) * complex(wigner_3j(Jprime, 1, Jprime, -Omegaprime, -q, 0)))

                #Matrix element for T(I)T(J)
                term2 = ((-1)**(mJprime+J-Omegaprime+I1-m1-q)*np.sqrt(J*(J+1)*(2*J+1)**2*(2*Jprime+1)*I1*(I1+1)
                                                                                   *(2*I1+1))
                        *complex(wigner_3j(Jprime,1,J,-mJprime,mJprime-mJ,mJ)) * complex(wigner_3j(Jprime,1,J,-Omegaprime,-q,0))
                        *complex(wigner_3j(J,1,J,0,-q,Omega)) * complex(wigner_3j(I1,1,I1,-m1prime, m1prime-m1, m1)))

                amp = c_Tl_p *0.5*(term1+term2)

                basis_state = UncoupledBasisState(Jprime, mJprime, I1prime, m1prime, I2prime, m2prime, Omegaprime)

                if amp != 0:
                        data.append((amp, basis_state))
                    
    return State(data)

### Electron magnetic hyperfine operator
Since the B state is a triplet pi state, it has electron magnetic hyperfine structure. The coupling is described by the Hamiltonian $ H_{\mathrm{mhf}} = a\, \mathbf{I} \cdot \mathbf{L} + b\, \mathbf{I} \cdot \mathbf{S} +c \,I_z\, S_z  $ which reduces to $ H_{\mathrm{mhf}}^{eff} =  [a \,L_z + (b+c)\, S_z]\,I_z = h_\Omega \, I_z $ since the raising and lowering operators for L and S (electron orbital and spin angular momentum) only couple different electronic states which are very far in energy and thus the effect of the off-diagonal elements is strongly suppressed. The z-direction in $I_z$ is along the internuclear axis of the molecule and therefore when evaluating the matrix elements one needs to rotate the spin operator to the lab fixed frame. This results in non-zero matrix elements between states with different J.

In the uncoupled basis these matrix elements are given by

\begin{equation*}
\mathrm{<J, m_J, \Omega, I_1, m_1, I_2, m_2 |\, I_z(Tl)\, | J', m'_J, \Omega', I'_1, m'_1, I'_2, m'_2>} \\
= - 
\begin{pmatrix}
J & 1 & J' \\
-\Omega & 0 & \Omega'
\end{pmatrix}
\lbrack (2J+1)(2J'+1)I_1(I_1+1)(2I_1+1) \rbrack^{\frac{1}{2}} \delta_{I_2,I'_2} \delta_{m_2,m'_2} \\
\sum_{p=\,-1}^{+1} (-1)^{p-m_J+I_1-m_1}
\begin{pmatrix}
J & 1 & J' \\
-m_J & -p & m'_J
\end{pmatrix}
\begin{pmatrix}
I_1 & 1 & I'_1 \\
-m_1 & p & m'_1
\end{pmatrix}
\end{equation*}

for Tl and 

\begin{equation*}
\mathrm{<J, m_J, \Omega, I_1, m_1, I_2, m_2 |\, I_z(Tl)\, | J', m'_J, \Omega', I'_1, m'_1, I'_2, m'_2>} \\
= - 
\begin{pmatrix}
J & 1 & J' \\
-\Omega & 0 & \Omega'
\end{pmatrix}
\lbrack (2J+1)(2J'+1)I_2(I_2+1)(2I_2+1) \rbrack^{\frac{1}{2}} \delta_{I_1,I'_1} \delta_{m_1,m'_1} \\
\sum_{p=\,-1}^{+1} (-1)^{p-m_J+I_2-m_2}
\begin{pmatrix}
J & 1 & J' \\
-m_J & -p & m'_J
\end{pmatrix}
\begin{pmatrix}
I_2 & 1 & I'_2 \\
-m_2 & p & m'_2
\end{pmatrix}
\end{equation*}

for F

In [14]:
def H_mhf_Tl(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    Omega = psi.Omega
    
    #I1, I2 and m2 must be the same for non-zero matrix element
    I2prime = I2
    m2prime = m2
    I1prime = I1
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over the possible values of quantum numbers for which the matrix element can be non-zero
    #Need Jprime = J+1 ... |J-1|
    for Jprime in np.arange(np.abs(J-1), J+2):
        #Evaluate the part of the matrix element that is common for all p
        common_coefficient = h1_Tl*wigner_3j(J, 1, Jprime, -Omega, 0, Omega)*np.sqrt((2*J+1)*(2*Jprime+1)*I1*(I1+1)*(2*I1+1))
        
        #Loop over the spherical tensor components of I1:
        for p in np.arange(-1,2):
            #To have non-zero matrix element need mJ-p = mJprime
            mJprime = mJ + p
            
            #Also need m2 - p = m2prime
            m1prime = m1 - p
            
            #Check that mJprime and m2prime are physical
            if np.abs(mJprime) <= Jprime and np.abs(m1prime) <= I1prime:
                #Calculate rest of matrix element
                p_factor = ((-1)**(p-mJ+I1-m1-Omega)*wigner_3j(J, 1, Jprime, -mJ, -p, mJprime)
                               *wigner_3j(I1, 1, I1prime, -m1, p, m1prime))
                               
                amp = Omega*common_coefficient*p_factor
                basis_state = UncoupledBasisState(Jprime, mJprime, I1prime, m1prime, I2prime, m2prime, psi.Omega)
                if amp != 0:
                    data.append((amp, basis_state))
    
    return State(data)

In [15]:
def H_mhf_F(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    Omega = psi.Omega
    
    #I1, I2 and m1 must be the same for non-zero matrix element
    I1prime = I1
    m1prime = m1
    I2prime = I2
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over the possible values of quantum numbers for which the matrix element can be non-zero
    #Need Jprime = J+1 ... |J-1|
    for Jprime in np.arange(np.abs(J-1), J+2):
        #Evaluate the part of the matrix element that is common for all p
        common_coefficient = h1_F*wigner_3j(J, 1, Jprime, -Omega, 0, Omega)*np.sqrt((2*J+1)*(2*Jprime+1)*I2*(I2+1)*(2*I2+1))
        
        #Loop over the spherical tensor components of I2:
        for p in np.arange(-1,2):
            #To have non-zero matrix element need mJ-p = mJprime
            mJprime = mJ + p
            
            #Also need m2 - p = m2prime
            m2prime = m2 - p
            
            #Check that mJprime and m2prime are physical
            if np.abs(mJprime) <= Jprime and np.abs(m2prime) <= I2prime:
                #Calculate rest of matrix element
                p_factor = ((-1)**(p-mJ+I2-m2-Omega)*wigner_3j(J, 1, Jprime, -mJ, -p, mJprime)
                               *wigner_3j(I2, 1, I2prime, -m2, p, m2prime))
                               
                amp = Omega*common_coefficient*p_factor
                basis_state = UncoupledBasisState(Jprime, mJprime, I1prime, m1prime, I2prime, m2prime, psi.Omega)
                if amp != 0:
                    data.append((amp, basis_state))
        
        
    return State(data)

The overall field-free Hamiltonian for the B state is

In [16]:
def Hff_B(psi):
    return Hrot_B(psi) + H_mhf_Tl(psi) + H_mhf_F(psi) + H_c1p(psi) + H_LD(psi)

### Zeeman Hamiltonian

In order to separate the task of finding the matrix elements and the eigenvalues, the Hamiltonian

$$H^\text{Z}=-\frac{\mu_J}{J}(\vec J\cdot\vec B)-\frac{\mu_1}{I_1}(\vec I_1\cdot\vec B)-\frac{\mu_2}{I_2}(\vec I_2\cdot\vec B)$$

is best split into three matrices:

$$H^\text{Z}=B_xH^\text{Z}_x+B_yH^\text{Z}_y+B_zH^\text{Z}_z,$$

where

$$ H^\text{Z}_x = -\frac{\mu_J}{J}J_x -\frac{\mu_1}{I_1}I_{1x} -\frac{\mu_2}{I_2}I_{2x} $$
$$ H^\text{Z}_y = -\frac{\mu_J}{J}J_y -\frac{\mu_1}{I_1}I_{1y} -\frac{\mu_2}{I_2}I_{2y} $$
$$ H^\text{Z}_z = -\frac{\mu_J}{J}J_z -\frac{\mu_1}{I_1}I_{1z} -\frac{\mu_2}{I_2}I_{2z} $$

Note that we are using the convention $\mu_1=\mu_\text{Tl}$ and $\mu_2=\mu_\text{F}$. The terms involving division by $J$ are only valid for states with $J\ne0$ (of course!).

Thus we can write the Zeeman functions for the X state:

In [17]:
def HZx_X(psi):
    if psi.J != 0:
        return -mu_J/psi.J*Jx(psi) - mu_Tl/psi.I1*I1x(psi) - mu_F/psi.I2*I2x(psi)
    else:
        return -mu_Tl/psi.I1*I1x(psi) - mu_F/psi.I2*I2x(psi)

def HZy_X(psi):
    if psi.J != 0:
        return -mu_J/psi.J*Jy(psi) - mu_Tl/psi.I1*I1y(psi) - mu_F/psi.I2*I2y(psi)
    else:
        return -mu_Tl/psi.I1*I1y(psi) - mu_F/psi.I2*I2y(psi)
    
def HZz_X(psi):
    if psi.J != 0:
        return -mu_J/psi.J*Jz(psi) - mu_Tl/psi.I1*I1z(psi) - mu_F/psi.I2*I2z(psi)
    else:
        return -mu_Tl/psi.I1*I1z(psi) - mu_F/psi.I2*I2z(psi)

For the B state, only defining for B-field along z for now, we have

In [ ]:
def HZx_B(psi):
    # TODO
    return psi

def HZy_B(psi):
    # TODO
    return psi

In [18]:
def HZz_B(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    Omega = psi.Omega
    S = 1
    
    #The other state must have the same value for I1,m1,I2,m2,mJ and Omega
    I1prime = I1
    m1prime = m1
    I2prime = I2
    m2prime = m2
    Omegaprime = Omega
    mJprime = mJ
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over possible values of Jprime
    for Jprime in range(np.abs(J-1),J+2):
        
        #Electron orbital angular momentum term
        L_term = (gL * Omega *np.sqrt((2*J+1)*(2*Jprime+1)) * (-1)**(mJprime-Omegaprime)
                  * complex(wigner_3j(Jprime,1,J,-mJprime,0,mJ)) * complex(wigner_3j(Jprime,1,J,-Omegaprime,0,Omega)))
        
        #Electron spin term
        S_term = (gS * np.sqrt((2*J+1)*(2*Jprime+1)) * (-1)**(mJprime-Omegaprime)
                  * complex(wigner_3j(Jprime,1,J,-mJprime,0,mJ)) * complex(wigner_3j(Jprime,1,J,-Omegaprime,0,Omega))
                  * (-1)**(S)*complex(wigner_3j(S,1,S,0,0,0)) * np.sqrt(S*(S+1)*(2*S+1)))
        
        amp = L_term+S_term
        basis_state = UncoupledBasisState(Jprime, mJprime, I1prime, m1prime, I2prime, m2prime, Omegaprime)
        
        
        if amp != 0:
            data.append((amp, basis_state))
                  
                  
    return State(data)    

### Stark Hamiltonian

Again splitting the Hamiltonian into the three spatial components, we have

$$H^\text{S}=-\vec d\cdot\vec E
= E_xH^\text{S}_x+E_yH^\text{S}_y+E_zH^\text{S}_z
= e(E_{-1}r_{-1} + E_{0}r_{0} + E_{1}r_{1})
$$

The matrix elements are given by 

$$
\langle J', m_J', \Omega |d_p| J, m_J, \Omega\rangle = \langle d \rangle (-1)^{m_J'- \Omega'}\left[(2J+1)(2J'+1)\right]^{1/2} \mathrm{ThreeJ(J',1,J,-m_j',p,m_J) }\mathrm{ThreeJ(J',1',J,-\Omega',\Omega'-\Omega,\Omega)}
$$

These $d_p$ operators can be written in Python as the functions:

In [19]:
def d_p(p,psi):
    #Find the quantum numbers of the input state
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    Omega = psi.Omega
    
    #The other state must have the same value for I1,m1,I2,m2,mJ and Omega
    I1prime = I1
    m1prime = m1
    I2prime = I2
    m2prime = m2
    Omegaprime = Omega
    mJprime = mJ+p
    q = Omegaprime - Omega
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over possible values of Jprime
    for Jprime in range(np.abs(J-1),J+2):
        amp = ((-1)**(mJprime - Omegaprime) * np.sqrt((2*J+1)*(2*Jprime+1))
               * complex(wigner_3j(Jprime,1,J,-mJprime,p, mJ)) * complex(wigner_3j(Jprime,1,J,-Omegaprime,q, Omega)))

        basis_state = UncoupledBasisState(Jprime, mJprime, I1prime, m1prime, I2prime, m2prime, Omegaprime)


        if amp != 0:
            data.append((amp, basis_state))

                  
    return State(data)    

In terms of the operators

$$
R_{\pm}\equiv \mp\frac{x\pm iy}{\sqrt2r} = 2\sqrt{\frac{\pi}{3}}Y_1^{\pm M} \\
R_{0}\equiv \frac{z}{r} = 2\sqrt{\frac{\pi}{3}}Y_1^{0}
$$
and the molecular dipole moment $d_\text{TlF}$, the three Stark Hamiltonians are

$$
\begin{align}
H^\text{S}_x&=-d_\text{TlF}(R^{-1}_1-R^1_1)/\sqrt2\\
H^\text{S}_y&=-d_\text{TlF}i(+R^{-1}_1+R^1_1)/\sqrt2\\
H^\text{S}_z&=-d_\text{TlF}R^0_1
\end{align}
$$

In Python:

In [20]:
def HSx(psi):
    return -D_TlF * ( d_p(-1,psi) - d_p(+1,psi) ) / np.sqrt(2)

def HSy(psi):
    return -D_TlF * 1j * ( d_p(-1,psi) + d_p(+1,psi) ) / np.sqrt(2)

def HSz(psi):
    return -D_TlF * d_p(0,psi)

### Finding the matrix elements

Define a function to evaluate the numerical matrix elements of a given operator `H` over a given basis `QN`:

In [21]:
def HMatElems(H, QN):
    result = np.zeros((len(QN), len(QN)), dtype=complex)
    for i,a in tqdm(enumerate(QN), total=len(QN)):
        for j,b in enumerate(QN):
            result[i,j] = (1*a)@H(b)
    return result

Write down the basis as a list of `UncoupledBasisState` components, and evaluate the X-state matrix elements for the Hamiltonian operators, storing the result of the calculation as Python `pickle`s:

In [22]:
QN_X = np.array([UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2)
      for J in range(Jmax+1)
      for mJ in range(-J,J+1)
      for m1 in np.arange(-I_Tl,I_Tl+1)
      for m2 in np.arange(-I_F,I_F+1)])

with open("data/1_Hamiltonians/X.pickle", 'wb') as f:
    pickle.dump(
        {
            "Hff" :  HMatElems(Hff_X, QN_X),
            "HSx" :  HMatElems(HSx, QN_X),
            "HSy" :  HMatElems(HSy, QN_X),
            "HSz" :  HMatElems(HSz, QN_X),
            "HZx" :  HMatElems(HZx_X, QN_X),
            "HZy" :  HMatElems(HZy_X, QN_X),
            "HZz" :  HMatElems(HZz_X, QN_X),
        },
        f
    )

Similarly, for the B state we have:

In [23]:
QN_B = []
for Omega in Omegas:
    for J in np.arange(Jmin, Jmax+1):
        for m1 in np.arange(-I_Tl,I_Tl+1):
            for m2 in np.arange(-I_F,I_F+1):
                #mJ, m1 and m2 values all sum to zero so mF = 0
                mJ = 0-m1-m2
                if np.abs(mJ) <= J:
                    QN_B.append(UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2, Omega))
                    
with open("data/1_Hamiltonians/B.pickle", 'wb') as f:
    pickle.dump(
        {
            "Hff" : HMatElems(Hff_B, QN_B),
            "HSx" : HMatElems(HSx, QN_B),
            "HSy" : HMatElems(HSy, QN_B),
            "HSz" : HMatElems(HSz, QN_B),
            "HZx" : HMatElems(HZx_B, QN_B),
            "HZy" : HMatElems(HZy_B, QN_B),
            "HZz" : HMatElems(HZz_B, QN_B),
        },
        f
    )

NameError: name 'c_Tl_p' is not defined

### Hamiltonian functions $H = H(E,B)$

Define a function that imports the matrix elements calculated above, and returns a function mapping from external lab fields $E, B$ to a Hamiltonian matrix:

In [25]:
def import_Hamiltonian(fname):
    # import matrix elements
    with open(fname, 'rb') as f:
        H = pickle.load(f)
    
    # return the mapping
    return lambda E,B: 2*np.pi*(H["Hff"] + \
        E[0]*H["HSx"]  + E[1]*H["HSy"] + E[2]*H["HSz"] + \
        B[0]*H["HZx"]  + B[1]*H["HZy"] + B[2]*H["HZz"])